In [1]:
import pandas as pd
import geopandas as gpd
import json
import os
from tqdm import tqdm

In [2]:
list_files = os.listdir('data_raw/sau_data')

In [3]:
gdf_sau = []
for file in tqdm(list_files):
    try:
        gdf_sauk = gpd.read_file(f'data_raw/sau_data/{file}')
        gdf_sau.append(gdf_sauk)
    except Exception as error:
        print(error)
gdf_sau = pd.concat(gdf_sau)

 14%|█▍        | 9/64 [00:15<01:28,  1.61s/it]

'data_raw/sau_data/.DS_Store' not recognized as a supported file format.


100%|██████████| 64/64 [03:34<00:00,  3.35s/it]


In [7]:
gdf_sau[['pk', 'launame', 'iso3', 'table_name', 'geo_level', 'fuacode',
       'fuaname_en', 'citycode', 'cityname_en', 'tl2_id', 'tl3_id',
       'tl2_name_en', 'tl3_name_en']].to_csv('../oecd-municipal-atlas-pm-data/data/data_geo_names_map.csv', index=False)

In [13]:
# gdf_sau[['pk', 'launame', 'iso3', 'geo_level', 'year_ref_geo', 'fuacode',
#        'fuaname_en', 'citycode', 'cityname_en', 'tl2_id', 'tl3_id',
#        'tl2_name_en', 'tl3_name_en', 'geometry']].to_file('../data_raw/sau.geojson')

In [6]:
# os.makedirs("sau_data", exist_ok=True)

In [18]:
# cols_excl_list = ['pk', 'launame', 'iso3', 'geo_level', 'year_ref_geo',
#                   'fuacode', 'fuaname_en', 'citycode', 'cityname_en',
#                   'tl2_id', 'tl3_id', 'tl2_name_en', 'tl3_name_en', 
#                   'geometry']

In [19]:
# for col in tqdm(gdf_sau.columns):
#     if col not in cols_excl_list:
#         os.makedirs(f"data/sau_data/{col}", exist_ok=True)
#         gdf_sau[['pk', col]].to_csv(f"data/sau_data/{col}/latest.csv", index=False)

100%|██████████| 128/128 [00:55<00:00,  2.31it/s]


# Build geo references JSON files

In [8]:
df_sau_names = pd.read_csv('../oecd-municipal-atlas-pm-data/data/data_geo_names_map.csv')

In [9]:
table_name_list = gdf_sau.table_name.unique()

In [10]:
os.makedirs('../oecd-municipal-atlas-pm-data/data/geo_names/sau_names', exist_ok=True)

In [11]:
tl1_names = df_sau_names[['iso3']].drop_duplicates()
tl1_names.reset_index(drop=True, inplace=True)

json_string = json.dumps(tl1_names.to_dict('records'))

# Save the JSON string to a file
# with open("geo_names/tl1_names_map.json", "w") as json_file:
#     json_file.write(json_string)

In [12]:
tl2_names = df_sau_names[['iso3', 'tl2_id', 'tl2_name_en']].drop_duplicates()
tl2_names.reset_index(drop=True, inplace=True)
tl2_names.dropna(inplace=True)

json_string = json.dumps(tl2_names.to_dict('records'))

# Save the JSON string to a file
with open("../oecd-municipal-atlas-pm-data/data/geo_names/tl2_names_map.json", "w") as json_file:
    json_file.write(json_string)

In [13]:
tl3_names = df_sau_names[['iso3', 'tl3_id', 'tl3_name_en']].drop_duplicates()
tl3_names.reset_index(drop=True, inplace=True)
tl3_names.dropna(inplace=True)

json_string = json.dumps(tl3_names.to_dict('records'))

# Save the JSON string to a file
with open("../oecd-municipal-atlas-pm-data/data/geo_names/tl3_names_map.json", "w") as json_file:
    json_file.write(json_string)

In [14]:
fua_names = df_sau_names[['iso3', 'fuacode', 'fuaname_en']].drop_duplicates()
fua_names.reset_index(drop=True, inplace=True)
fua_names.dropna(inplace=True)

json_string = json.dumps(fua_names.to_dict('records'))

# Save the JSON string to a file
with open("../oecd-municipal-atlas-pm-data/data/geo_names/fua_names_map.json", "w") as json_file:
    json_file.write(json_string)

In [15]:
city_names = df_sau_names[['iso3', 'citycode', 'cityname_en']].drop_duplicates()
city_names.reset_index(drop=True, inplace=True)
city_names.dropna(inplace=True)

json_string = json.dumps(city_names.to_dict('records'))

# Save the JSON string to a file
with open("../oecd-municipal-atlas-pm-data/data/geo_names/city_names_map.json", "w") as json_file:
    json_file.write(json_string)

In [16]:
for table_name in table_name_list:
    print(table_name)
    iso3 = table_name.split('_')[0].upper()

    json_data = []
    
    for k, row in tl2_names[tl2_names.iso3 == iso3].iterrows(): 
        json_data_k_children = df_sau_names[(df_sau_names.tl2_id == row['tl2_id']) & 
                                            (df_sau_names.table_name == table_name)][['pk', 'launame', 'tl2_name_en', 'tl3_name_en', 'fuaname_en', 'cityname_en']].fillna('')
        json_data_k_children = json_data_k_children.rename(columns={'pk': 'id', 'launame': 'text'})
        json_data_k_children = json_data_k_children.to_dict('records')
        json_data_k = {
            "tl2_id": row["tl2_id"],  
            "tl2_name_en": row["tl2_name_en"],  
            "children": json_data_k_children
        }
        json_data.append(json_data_k)
    
    json_string = json.dumps(json_data)
    # Save the JSON string to a file
    with open(f"../oecd-municipal-atlas-pm-data/data/geo_names/sau_names/{table_name}.json", "w") as json_file:
        json_file.write(json_string)

lva_sau2_2022
ltu_sau3_2020
pol_sau4_2024
nzl_sau3_2022
prt_sau3_2021
nzl_sau4_2022
nld_sau3_2023
lux_sau3_2021
deu_postcode_2022
lva_sau3_2022
bel_sau3_2022
cri_sau4_2022
dnk_sau3_2022
usa_sau3_2022
dnk_postcode_2024
hrv_sau3_2020
irl_sau2_2022
deu_sau3_2022
usa_postcode_2020
prt_sau2_2023
irl_sau3_2022
chl_sau3_2017
cyp_sau3_2020
fin_sau2_2022
svk_sau3_2021
tur_sau4_2022
che_sau3_2021
jpn_sau3_2020
mex_sau3_2021
rou_sau3_2020
can_sau3_2021
svn_sau3_2021
bgr_sau3_2020
cze_sau3_2022
gbr_sau3_2022
prt_sau2_2018
mex_sau2_2021
grc_sau4_2015
fin_sau3_2022
irl_sau4_2019
aus_sau4_2023
isl_sau3_2023
est_sau3_2023
fra_sau3_2022
cri_sau3_2018
nor_sau3_2022
usa_place_2022
swe_sau3_2021
aus_sau3_2021
aut_sau3_2022
gbr_postcode_2017
kor_sau3_2023
ita_sau3_2022
hun_sau3_2020
esp_sau3_2021
fra_sau2_2022
hun_sau2_2016
grc_sau3_2020
col_sau3_2021
pol_sau3_2021
kor_sau2_2023
mlt_sau3_2020
usa_cousub_2022


# Creating JSON database

In [27]:
# gdf_sau = gpd.read_file('data_raw/data_geo.geojson')

In [28]:
#df_sau = gdf_sau.drop(columns='geometry')

In [17]:
os.makedirs('../oecd-municipal-atlas-pm-data/data/geo_data', exist_ok=True)

In [18]:
for table_name in table_name_list:
    print(table_name)
    os.makedirs(f'../oecd-municipal-atlas-pm-data/data/geo_data/{table_name}', exist_ok=True)
    for k, row in gdf_sau[gdf_sau.table_name == table_name].iterrows():
        pk = row['pk'].replace('/', '-')
        gpd.GeoDataFrame([row]).to_file(f'../oecd-municipal-atlas-pm-data/data/geo_data/{table_name}/{pk}.json', driver="GeoJSON")
        # row_dict = row.to_dict()
        # row_dict = {k: None if pd.isna(v) else v for k, v in row_dict.items()}
        
        # json_string = json.dumps(row_dict)
        # # Save the JSON string to a file
        # with open(f'data/geo_data/{iso3}/{pk}.json', "w") as json_file:
        #     json_file.write(json_string)

lva_sau2_2022
ltu_sau3_2020
pol_sau4_2024
nzl_sau3_2022
prt_sau3_2021
nzl_sau4_2022
nld_sau3_2023
lux_sau3_2021
deu_postcode_2022
lva_sau3_2022
bel_sau3_2022
cri_sau4_2022
dnk_sau3_2022
usa_sau3_2022
dnk_postcode_2024
hrv_sau3_2020
irl_sau2_2022
deu_sau3_2022
usa_postcode_2020
prt_sau2_2023
irl_sau3_2022
chl_sau3_2017
cyp_sau3_2020
fin_sau2_2022
svk_sau3_2021
tur_sau4_2022
che_sau3_2021
jpn_sau3_2020
mex_sau3_2021
rou_sau3_2020
can_sau3_2021
svn_sau3_2021
bgr_sau3_2020
cze_sau3_2022
gbr_sau3_2022
prt_sau2_2018
mex_sau2_2021
grc_sau4_2015
fin_sau3_2022
irl_sau4_2019
aus_sau4_2023
isl_sau3_2023
est_sau3_2023
fra_sau3_2022
cri_sau3_2018
nor_sau3_2022
usa_place_2022
swe_sau3_2021
aus_sau3_2021
aut_sau3_2022
gbr_postcode_2017
kor_sau3_2023
ita_sau3_2022
hun_sau3_2020
esp_sau3_2021
fra_sau2_2022
hun_sau2_2016
grc_sau3_2020
col_sau3_2021
pol_sau3_2021
kor_sau2_2023
mlt_sau3_2020
usa_cousub_2022
